In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

# Load data
df = pd.read_csv("heart_failure_clinical_records_dataset.csv")

# Features and target
X = df.drop("DEATH_EVENT", axis=1)
y = df["DEATH_EVENT"]

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Initial SVC model
svc = SVC()
svc.fit(X_train, y_train)
print("Initial SVC Model Performance:\n", classification_report(y_test, svc.predict(X_test)))

# GridSearchCV for best hyperparameters
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 0.01, 0.001],
    'kernel': ['rbf', 'linear']
}
grid = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)
print("\nBest Params:", grid.best_params_)

# Evaluate best model
best_svc = grid.best_estimator_
print("\nGridSearchCV SVC Performance:\n", classification_report(y_test, best_svc.predict(X_test)))

# Apply SMOTE
sm = SMOTE(random_state=42)
X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)

svc_smote = SVC(**grid.best_params_)
svc_smote.fit(X_train_sm, y_train_sm)
y_pred = svc_smote.predict(X_test)
print("\nSVC after SMOTE:\n", classification_report(y_test, y_pred))


Initial SVC Model Performance:
               precision    recall  f1-score   support

           0       0.78      0.93      0.84        41
           1       0.73      0.42      0.53        19

    accuracy                           0.77        60
   macro avg       0.75      0.67      0.69        60
weighted avg       0.76      0.77      0.75        60


Best Params: {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}

GridSearchCV SVC Performance:
               precision    recall  f1-score   support

           0       0.78      0.95      0.86        41
           1       0.80      0.42      0.55        19

    accuracy                           0.78        60
   macro avg       0.79      0.69      0.70        60
weighted avg       0.79      0.78      0.76        60


SVC after SMOTE:
               precision    recall  f1-score   support

           0       0.83      0.85      0.84        41
           1       0.67      0.63      0.65        19

    accuracy                          